In [34]:
import pandas as pd
import pylab as pl
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import re
import networkx as nx
import csv
import json

import plotly.plotly as py
import plotly.tools as tls
from plotly.graph_objs import *

mpl_fig = plt.figure()

In [58]:
df = pd.read_csv(data_path+'gcc_user_info.csv')

In [62]:
df.describe()

,4
count,69194.000000
mean,5174127.673917
std,4233122.902293
min,6.000000
25%,1161685.500000
50%,4607535.000000
75%,8264743.000000
max,14046314.000000


In [35]:
data_path = r"/Users/christopherallison/Documents/Coding/Data/GCconnex_collab/"

In [51]:
nodes = []
edges = []
details = {}
errors = []

In [52]:
# Read CSV of UID and names and create nodes for each user

with open(data_path+"gcc_user_info.csv", "r") as f:
    reader = csv.reader(f, delimiter=',')

    for row in reader:
        ampersand = row[1].find("@")
        uid = int(row[0])
        name = row[1][0:ampersand]
        department = row[1][ampersand + 1:]
        details[uid] = {'name':name, 'department':department}
        nodes.append(row[0])
        print(uid, name, department)


4 jean.richard tpsgc-pwgsc.gc.ca
6 Charles.degrasse tbs-sct.gc.ca
7 Karl.Ghiara hc-sc.gc.ca
19 joel.racicot canada.ca
22 bruno.grossmann international.gc.ca
23 plpilon justice.gc.ca
42 jeff.braybrook inspection.gc.ca
58 Laura.Simpson hc-sc.gc.ca
59 Sean.Jones tbs-sct.gc.ca
70 Rick.Brouzes tbs-sct.gc.ca
75 thomas.kearney tbs-sct.gc.ca
106 mike.just tpsgc-pwgsc.gc.ca
139 douglas.bastien csps-efpc.gc.ca
145 Ian.Ketcheson aandc.gc.ca
146 David.Hume ainc-inac.gc.ca
151 paula.colwell csps-efpc.gc.ca
162 carlos.chan tpsgc-pwgsc.gc.ca
165 colin.mckay priv.gc.ca
195 don.daniels tpsgc-pwgsc.gc.ca
196 chuck.shawcross ec.gc.ca
205 alvin.hamel tpsgc-pwgsc.gc.ca
218 shawn.pigeon tpsgc-pwgsc.gc.ca
222 eric.morrissette hrsdc-rhdcc.gc.ca
224 Peter.Cowan ceaa-acee.gc.ca
278 Paul.Jackson tbs-sct.gc.ca
304 leszek.nowosielski csps-efpc.gc.ca
307 bill.ranger tpsgc-pwgsc.gc.ca
335 james.steele tpsgc-pwgsc.gc.ca
336 david.delorey tpsgc-pwgsc.gc.ca
342 Barbara.Dundas tbs-sct.gc.ca
352 Amanda.Stringer tbs-sct.g

In [53]:
details

{131073: {'department': 'NRCan.gc.ca', 'name': 'Cynthia.Ng'},
 9153195: {'department': 'ec.gc.ca', 'name': 'bijan.rasti'},
 4: {'department': 'tpsgc-pwgsc.gc.ca', 'name': 'jean.richard'},
 6: {'department': 'tbs-sct.gc.ca', 'name': 'Charles.degrasse'},
 7: {'department': 'hc-sc.gc.ca', 'name': 'Karl.Ghiara'},
 4980745: {'department': 'csc-scc.gc.ca', 'name': 'wendy.marthone'},
 8519690: {'department': 'hc-sc.gc.ca', 'name': 'charlene.libert-scott'},
 8344919: {'department': 'justice.gc.ca', 'name': 'michel.brazeau'},
 3276814: {'department': 'infc.gc.ca', 'name': 'alexaida.mendoza'},
 393232: {'department': 'csc-scc.gc.ca', 'name': 'Lindsay.Holloway'},
 19: {'department': 'canada.ca', 'name': 'joel.racicot'},
 4587540: {'department': 'aadnc-aandc.gc.ca', 'name': 'marc.fonda'},
 174766: {'department': 'cmhc-schl.gc.ca', 'name': 'lcornell'},
 22: {'department': 'international.gc.ca', 'name': 'bruno.grossmann'},
 23: {'department': 'justice.gc.ca', 'name': 'plpilon'},
 3276824: {'departme

In [56]:
# Read CSV of user names and departments and add those to nodes

with open(data_path+"gcconnex_users_activity_joined.csv", 'r') as f:
    reader = csv.reader(f, delimiter=',')
    next(reader) #skip first line
    
    for row in reader:
        try:
            details[int(row[0])]['active'] = row[3] # last active date
            details[int(row[0])]['end'] = "01-09-2015"
            details[int(row[0])]['start'] = row[4] #account created
            #details[row[0]]['updated'] = row[5]
        except KeyError:
            ampersand = row[1].find("@")
            uid = int(row[0])
            name = row[1][0:ampersand]
            department = row[1][ampersand + 1:]
            details[uid] = {'name':name, 'department':department}
            details[int(row[0])]['active'] = row[3] # last active date
            details[int(row[0])]['end'] = "01-09-2015"
            details[int(row[0])]['start'] = row[4] #account created
            

In [69]:
# Read CSV blogs and create edges for connections

with open(data_path+"gccResults_blogs.csv", 'r') as f:
    reader = csv.reader(f, delimiter=',')
    next(reader)
    
    for row in reader:
        if len(row) > 1:
            blog_data = row[0].split(' ')
            blog = blog_data[0]
            author = blog_data[1]
            date_created = blog_data[2]
            #print(blog, author, date_created)
        
            for c in row[1:]: # Get comments (2+ columns)
                comment = c.split()
                commenter = comment[1]
                comment_date = comment[2]
                edges.append([commenter, author, comment_date])
            

In [70]:
edges

[['145', '19', '02/01/2009'],
 ['19', '19', '05/01/2009'],
 ['6', '19', '05/01/2009'],
 ['19', '19', '06/01/2009'],
 ['19', '23', '07/01/2009'],
 ['23', '23', '07/01/2009'],
 ['106', '304', '23/01/2009'],
 ['562', '304', '24/02/2009'],
 ['362', '391', '30/01/2009'],
 ['537', '19', '09/02/2009'],
 ['19', '106', '03/02/2009'],
 ['106', '362', '05/02/2009'],
 ['562', '362', '09/02/2009'],
 ['304', '391', '06/02/2009'],
 ['37518', '537', '09/12/2009'],
 ['537', '537', '10/12/2009'],
 ['623', '362', '19/02/2009'],
 ['3552', '362', '28/05/2009'],
 ['41987', '501', '05/02/2010'],
 ['151', '75', '23/03/2009'],
 ['75', '75', '24/03/2009'],
 ['151', '75', '25/03/2009'],
 ['3712', '3712', '02/06/2009'],
 ['4141', '4284', '03/06/2009'],
 ['75', '4141', '20/09/2009'],
 ['25697', '4723', '03/09/2009'],
 ['3552', '4141', '12/06/2009'],
 ['4141', '4141', '12/06/2009'],
 ['8101', '4141', '19/06/2009'],
 ['4141', '4141', '19/06/2009'],
 ['46162', '4141', '05/03/2010'],
 ['4141', '4141', '05/03/2010'],
 

In [55]:
edge_dict = {}

for u in users:
    temp = []

    # Prep user-mentions
    try:
        um = users[u]['user_mentions'].split(" ")
    except AttributeError:
        um = []

    # Prep replies to users
    try:
        ur = users[u]['user_replies'].split(" ")
    except AttributeError:
        ur = []


    # Prep replies to tweets
    try:
        rtt = users[u]['replies'].split(" ")
    except AttributeError:
        rtt = []

    if rtt:
        rtt = [find_user(u, tweets) for u in rtt]

    # Prep quoted_status_id_str
    try:
        qis = users[u]['quoted_id_str'].split(" ")
    except AttributeError:
        qis = []

    if qis:
        qis = [find_user(u, tweets) for u in qis]


    # Create edge list keys and weight
    temp = um + ur + rtt + qis

    # Remove any empty elements in list
    for e in temp:

        # Remove any empty elements in list
        if str(e) == "" or str(e) == 'None':
            pass
        # Add edges to dict
        else:
            try:
                edge_dict[str([users[u]['screen_name'], str(e)]).lower()]['weight'] += 1

            except KeyError:
                edge_dict[str([users[u]['screen_name'], str(e)]).lower()] = {}
                edge_dict[str([users[u]['screen_name'], str(e)]).lower()]['node'] = users[u]['screen_name']
                edge_dict[str([users[u]['screen_name'], str(e)]).lower()]['target'] = str(e)
                edge_dict[str([users[u]['screen_name'], str(e)]).lower()]['weight'] = 1

# Add edges for @mentions and replies to users
for e in edge_dict:
    N.add_edge(edge_dict[e]['node'],
               edge_dict[e]['target'],
               weight=edge_dict[e]['weight'])

[['219682',
  'Bernie.Goodman@cra-arc.gc.ca',
  '28-11-2012',
  '06-12-2012',
  '28-11-2012',
  '06-12-2012'],
 ['2103048',
  'isabelle.corriveau@ps-sp.gc.ca',
  '14-04-2015',
  '29-05-2015',
  '17-09-2013',
  '13-07-2015'],
 ['2479649',
  'thompsonmm@dfo-mpo.gc.ca',
  '03-10-2013',
  '08-10-2013',
  '24-09-2013',
  '24-09-2013'],
 ['4159458',
  'ghislaine.dorval@cra-arc.gc.ca',
  '26-02-2015',
  '18-08-2014',
  '20-02-2014',
  '18-08-2014'],
 ['4171189',
  'patricia.gagnon@arc-cra.gc.ca',
  '25-02-2014',
  '25-02-2014',
  '25-02-2014',
  '25-02-2014'],
 ['4343361',
  'mihaela.tapuc@pch.gc.ca',
  '06-05-2015',
  '28-05-2015',
  '17-03-2014',
  '28-05-2015'],
 ['4897595',
  'dudley.maseko@justice.gc.ca',
  '02-02-2015',
  '15-07-2015',
  '30-05-2014',
  '15-07-2015'],
 ['8584673',
  'isabelle.corriveau@ps.gc.ca',
  '26-01-2015',
  '26-01-2015',
  '26-01-2015',
  '26-01-2015'],
 ['9129571',
  'isabelle.corriveau@ps-sp.gc.ca',
  '11-02-2015',
  '20-02-2015',
  '11-02-2015',
  '20-02-2015'

In [7]:
G=nx.DiGraph()

In [8]:
for k,v in details.items():
    G.add_node(k, label=v['name'], department=v['department'], activity=v['active'], start=v['start'], end=v['end'])
    
for colleague_link in edges:
    G.add_edge(colleague_link[0], colleague_link[1])

In [9]:
# Normally, you could graph this directly, but the graph is too big.  Don't run this.

#pos=nx.spring_layout(G)

In [10]:
# Or this

#nx.draw_networkx_nodes(G, pos)
#nx.draw_networkx_edges(G, pos)

In [11]:
# Or this

#plt.show()

In [12]:
# Export to JSON format
from networkx.readwrite import json_graph

d = json_graph.node_link_data(G)
json.dump(d, open('/Users/christopherallison/Documents/Coding/Python/Network Graph/force/GC_network_activity.json', 'w'))

In [13]:
# Write to Graphml format
nx.write_graphml(G,"GCconnex_users_activity_dynamic.graphml")

### Now open up Gephi and play with your data!